In [1]:
# import sys 
# sys.path.append('/home/aistudio/external-libraries')
#------------------------------调包-----------------------#
# import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
# import osmnx as ox
# import taxicab as tc  #导入taxicab包
import math
import random
from random import randint
#加载依赖库
import networkx as nx
from networkx.algorithms import approximation         #调用启发式算法的包
import PIL 
import numpy as np
from haversine import haversine
#四个常数，可以测试取最优结果，目前取1
c1=1
c2=1
c3=1
c4=1

In [2]:
# #获得嘉定区路网图
# G1=ox.graph.graph_from_xml(r"D:\大学学习资料汇总\交通工程专业\2022交通科技大赛\数据\sh.osm") #获取嘉定区数据
# G1 = ox.projection.project_graph(G1)   #进行投影
# # G_84 = ox.projection.project_graph(G1,to_crs='EPSG:4326')  

In [3]:
res=[0]
# res[-2]


In [4]:
# for i in range(len(G2.nodes)):
#     print(G2.nodes[i]['weight'])

In [5]:
#---------------------------------最大权重独立集函数--------------------------#
#初始化生成初始解
def Initialize(G):
    S=[]
    AddFreeNode(S,G)
    return S
            
#求解独立集权重的函数
def Weight(S,G):
    total=0
    if(len(S)):
        for i in S:
            total+=G.nodes[i]['weight']
    return total

#返回V-S的集合
def GET_V_S(S,G):
    V_S=[]
    if(S):
        for i in G.nodes:
            if(i not in S):
                V_S.append(i)
        return V_S
    else:
        return list(G.nodes)
#随机均匀添加节点
def AddFreeNode(S,G):
    V_S=GET_V_S(S,G)
    Isend=True
    while(Isend):
        k=randint(0,len(V_S)-1) #随机生成节点
        j=V_S[k]              #j为节点
        if(S==None):
            S.append(j)
            V_S.remove(j)
        else :
            Isadd=True          #判断是否添加节点
            for i in S:
                if((i,j) in G.edges or G.nodes[j]['weight']<0):       #说明节点不独立，或者权重为0，不添加
                    Isadd=False
                    break
            if(Isadd):
                S.append(j)
                V_S.remove(j)
        
        #下列代码为了判断S是否已经是最大集  
        Ending=True
        for i in V_S:
            Iexit=False
            for j in S:
                if((i,j) in G.edges()):
                    Iexit=True               #说明相连
                    break
            if(Iexit==False and G.nodes[i]['weight']>=0):    
                S.append(i)
                V_S.remove(i)
                Ending=False
                break
        if(Ending):
            Isend=False         
#扰动函数
def Perturb(k,S,G):
    S1=S.copy()
    V_S=GET_V_S(S,G)
    Se=[]
    i=0
    while(i<k):
        a=randint(0,len(V_S)-1)
        
        b=V_S[a]
        if(Se==None):
            Se.append(b)
            i+=1
        elif(b not in Se):
            Notadd=False
            for vertex in Se:
                if((b,vertex) in G.edges):
                    Notadd=True
                    break
            if(not Notadd):
                Se.append(b)
                i+=1
    #进行扰动
    RE=[]  #记录需要删除的元素,不能直接remove，会有bug
    for ve in Se:
        for v in S1:
            if((v,ve) in G.edges()):
                RE.append(v)
    #加入节点
    for v in RE:
        S1.remove(v)
    for v in Se:
        S1.append(v)
    AddFreeNode(S1,G)
    return S1

#利用邻域框架添加节点
def Firstimprovement(k,S,G):  #O(n2)
    V_S=GET_V_S(S,G)
    Vs=V_S.copy()
    S1=S.copy() 
    if k==1:  #w-1交换法:插入1个节点，删除与之相连的w个顶点
        for v in Vs:
            total=0   #记录权重，判断删除后效果
            a=[]  #记录节点
            for k in S1:
                if((v,k) in G.edges):
                    total+=G.nodes[k]['weight']
                    a.append(k)
            #如果权重满足条件的话
            if(G.nodes[v]['weight']-total>0):
                for k in a:
                    if(k in S1):
                        S1.remove(k)
                S1.append(v)
                break
        return S1
    else:         #1-2交换法   删除1个顶点，交换与之相连的顶点——   O(n3)
        for i in S1:
            count=0
            Solution=[]     #先求得可能加入的点集
            T_Solution=[]
            for j in V_S:
                if((i,j) in G.edges):
                    Solution.append(j)
            #找到与i相连的两个顶点,并且这两个与其他顶点不相连
            RE=[]
            for j in S1:
                if(j!=i):
                    for k in Solution:
                        if((k,j) in G.edges):
                            RE.append(k)
            for vertex in RE:
                Solution.remove(vertex)
            #若条件判断后节点不为空
            if(Solution):  #删除节点后不为空
                for k1 in range(len(Solution)):
                    for k2 in range(k1+1,len(Solution)):
                        #k1,k2只是下标，具体的还要得到节点值!!pay attention!
                        k1node=Solution[k1]    
                        k2node=Solution[k2]
                        if(((k1node,k2node) not in G.edges) and ((G.nodes[k1node]['weight']+G.nodes[k2node]['weight'])>=G.nodes[i]['weight'])):
                            T_Solution.append((k1node,k2node))
            #进行加入判断,如果满足权重要求，就加入
            if(T_Solution):
                S1.remove(i)
                S1.append(T_Solution[0][0])
                S1.append(T_Solution[0][1])
                break
        return S1 
#局部搜索算法


#Add FreeNnode in order to make max independent set
def LocalSearch(S,G):
    k=1
    while(k<=2):
        #这里出现了死循环
        S1=Firstimprovement(k,S,G)    #k=1采用1个插入方式，k=2采用(1,2)交换法 ,也就是两个邻域结构
        if(Weight(S1,G)<=Weight(S,G)):
            k+=1
        else:
            k=1
            S=S1
            AddFreeNode(S,G)          #inorder to make maximum set number
    return S

#接受函数
def Accept(S,S_,S1,i,local_best_w,G):
    if(Weight(S,G)<Weight(S1,G)):
        S=S1
        i=1
        if local_best_w<Weight(S,G):
            local_best_w=Weight(S,G)
            i=i-(len(S)/c2)
        if Weight(S_,G)<Weight(S,G):
            S_=S
            i=i-(len(S)/c3)
    elif(i<=(len(S)/c2)):
        i+=1
    else:
        local_best_w=Weight(S,G)
        S=Perturb(c4,S,G)
        i=1
    return (S,S_,i,local_best_w)
#局部迭代搜索函数
def ILS_VND(G,maxIter):
    '''
    传入图和最大迭代次数
    '''
    global c1,c2,c3,c4
    S0 = Initialize(G) # initial solution
    S = LocalSearch(S0, G) #current solution,通过本地搜索得到一个解
    S_=S
    local_best_w = Weight(S,G)  
    iter = 1
    i = 1
    while iter<maxIter:
        S1=Perturb(c1,S,G)       #扰动解决方案
        S1 = LocalSearch(S1, G)
        (S, S_, i, local_best_w) = Accept(S, S_, S1, i, local_best_w, G)
        iter = iter + 1
    return S_
#----------------------------------------------End---------------------------------------------#

In [6]:
# #15 15 3下的不同指标的对比
# res=np.zeros([4,4])
# for i in [1]:
#     GD1=Creat_Graph(D,15,3,i)
#     G2,KONG=ConALL(GD1)
#     print("Geneal")
#     SE=ILS_VND(G2,50)
#     d0=0 #绕行比的出行距离 与dis0区分开来
#     dis1=0
#     time1=0
#     cost1=0
#     for j in SE:
#         d0+=D.loc[G2.nodes[j]['key'][0],"行驶距离/m"]
#         d0+=D.loc[G2.nodes[j]['key'][1],"行驶距离/m"]
#         dis1+=G2.nodes[j]['another'][0]
#         time1+=G2.nodes[j]['another'][1]
#         cost1+=G2.nodes[j]['another'][2]
#         dis_jianshao[i]+=G2.nodes[j]['dis']
#         chezailv[i]+=G2.nodes[j]['dis1']
#         yanwu[i]+=G2.nodes[j]['yanwu']
#     car_num=Get_MinusCar(G2,D,SE)
#     res[i][0]=dis1/len(SE)
#     res[i][1]=time1/len(SE)
#     res[i][2]=cost1/len(SE)
#     res[i][3]=car_num/len(D)
#     raoxingjuli[i]=dis_jianshao[i]/len(SE)
#     raoxingbi[i]=dis_jianshao[i]/d0
#     pingchelv[i]+=(len(SE)*2/TRIP_NUM)
#     chezailv[i]=chezailv[i]/len(SE)
#     yanwu[i]=yanwu[i]/len(SE)
#     print(res[i])

In [7]:
# GET_V_S(S,G)

In [8]:
# G2.nodes[158]["weight"]

In [9]:
# G2.nodes[159]["weight"]

In [10]:
# G2.nodes[160]["weight"]+G2.nodes[159]["weight"]

In [11]:
# #----------------------------------进行数据的处理-------------------------------# 
# #导入数据
# TRIP_NUM = 100
# Data=pd.read_excel(r"D:\大学学习资料汇总\交通工程专业\2022交通科技大赛\数据\Data.xlsx")
# D=Data.iloc[0:5000] #取一百个数据尝试

In [12]:
Data=pd.read_excel(r"5000D.xlsx")
D=Data.iloc[0:5000:10] #取一百个数据尝试
#出行除0问题？
D=D.reset_index()

In [13]:
D

,index,Unnamed: 0,出发时间,到达时间,出发X_84,出发Y_84,到达X_84,到达Y_84,行驶距离/m,花费时间,Home_X,Home_Y
0,0,0,07:30:00,07:50:00,121.217303,31.292221,121.211738,31.264727,3958.457,20,121.221747,31.290239
1,10,10,08:00:00,09:00:00,121.218006,31.291656,121.445415,31.294686,29525.186,60,121.222449,31.289673
2,20,20,07:50:00,08:05:00,121.218049,31.290688,121.205473,31.278498,2715.731,15,121.222492,31.288704
3,30,30,18:00:00,18:30:00,121.253335,31.275876,121.216634,31.289105,4881.711,30,121.221079,31.287122
4,40,40,17:30:00,19:00:00,121.428834,31.286705,121.174745,31.284623,32662.713,90,121.179258,31.282703
...,...,...,...,...,...,...,...,...,...,...,...,...
495,4950,4950,06:50:00,07:05:00,121.306075,31.244145,121.345256,31.267581,13647.622,15,121.349140,31.265875
496,4960,4960,18:55:00,19:45:00,121.380723,31.267480,121.346793,31.267273,11351.993,50,121.349140,31.265875
497,4970,4970,07:55:00,08:10:00,121.335142,31.269039,121.346104,31.271208,1291.134,15,121.349140,31.265875
498,4980,4980,07:10:00,07:25:00,121.317149,31.257890,121.345635,31.267443,3814.624,15,121.349140,31.265875


In [14]:
for i in range(len(D)):
    if (D.loc[i,"行驶距离/m"]==0):
        D.drop(index=i,inplace=True)

In [15]:
TRIP_NUM=len(D)
D=D.reset_index()

In [16]:
#-------------距离和时间函数定义-------------------------#
def Get_Dis(e1,n1,e2,n2):      #计算所有路径距离,传入两个点的经度，纬度
        # O1=ox.distance.nearest_nodes(G_84,e1,n1)  #计算起点最近节点---------注：这里可以提前优化进行数据处理
        # O2=ox.distance.nearest_nodes(G_84,e2,n2)  #计算终点最近节点
        # if(nx.has_path(G_84,O1,O2)):
        #     Dis=nx.shortest_path_length(G_84,O1,O2,weight='length')
        # else:
        #否则就用经纬度进行近似计算
        lont1=(n1,e1)
        lont2=(n2,e2)
        Dis=haversine(lont1,lont2)*1000 #求出来默认是公里，转化成m
        return Dis
def cal_time(time1,time2):   #返回出行时间
    h1=time1.hour
    h2=time2.hour
    m1=time1.minute
    m2=time2.minute
    return (h2-h1)*60+m2-m1

#计算成本
def cost(dis,leap): #leap=0 不合乘的出行 leap=1 合乘的出行
    p0=[8,1.3,1]
    p1=[6.4,1.04,0.78,0.6,0.4,0.3]
    c=0
    dis=dis/1000
    if(leap==0):
        if(dis<=3):
            c=dis*p0[0]
        elif(dis<=15):
            c=3*p0[0]+(dis-3)*p0[1]
        else:
            c=3*p0[0]+12*p0[1]+(dis-15)*p0[2]
    else:
        if(dis<=3):
            c=dis*p1[0]
        elif(dis<=10):
            c=3*p1[0]+(dis-3)*p1[1]
        elif(dis<=15):
            c=3*p1[0]+7*p1[1]+(dis-10)*p1[2]
        elif(dis<=30):
            c=3*p1[0]+7*p1[1]+5*p1[2]+(dis-15)*p1[3]
        elif(dis<=500):
            c=3*p1[0]+7*p1[1]+5*p1[2]+15*p1[3]+(dis-15)*p1[4]
        else:
            c=3*p1[0]+7*p1[1]+5*p1[2]+15*p1[3]+485*p1[4]+(dis-500)*p1[5]
        #c=c*2
    return c

In [17]:
# #修订一下行驶距离
# for i in range(len(D)):
#     e1=D.loc[i,"出发X_84"]
#     n1=D.loc[i,"出发Y_84"]
#     e2=D.loc[i,"到达X_84"]
#     n2=D.loc[i,"到达Y_84"]
#     D.loc[i,'行驶距离/m']=Get_Dis(G_84,e1,n1,e2,n2)

In [18]:
# D.to_excel(r"D:\大学学习资料汇总\交通工程专业\2022交通科技大赛\数据\5000D.xlsx")

In [19]:
#建立路网，这里得到是GD1
def Creat_Graph(Data,time_limit,distance_limit,leap): #参数leap表示建立路网以何为权重 0为车辆数 1为路程 2为时间
    #创建图
    G=nx.Graph()
    G.add_nodes_from(Data.index)
    for i in range(len(Data)):
        for j in range(i+1,len(Data)):
                #当前出行的起终点
                lonlat1=(Data.loc[i,"Home_Y"],Data.loc[i,"Home_X"])
                lonlat2=(Data.loc[j,"Home_Y"],Data.loc[j,"Home_X"])
                Dis0=haversine(lonlat1,lonlat2) #家庭地址之间的距离利用正常经纬度算出来
                if(Dis0<distance_limit) : #是否满足社交距离
                    #2次出行的时间
                    O_i=Data.loc[i,"出发时间"]
                    D_i=Data.loc[i,"到达时间"]
                    O_j=Data.loc[j,"出发时间"]
                    D_j=Data.loc[j,"到达时间"]
                    if(D_j >O_i and  D_i>O_j):   #首先满足第一道时间约束
                        ti=Data.loc[i,"花费时间"]
                        tj=Data.loc[j,"花费时间"]
                        #计算时间差
                        delta_Begin=cal_time(O_i,O_j)
                        delta_End=cal_time(D_i,D_j)
                        #2次出行的起终点的84经纬度
                        #Get_Dis(G,e1,n1,e2,n2):      #计算所有路径距离,传入两个点的经度，纬度
                        #计算两个起点和两个终点的距离差 利用84坐标系
                        Dis1=Get_Dis(Data.loc[i,"出发X_84"],Data.loc[i,"出发Y_84"],Data.loc[j,"出发X_84"],Data.loc[j,"出发Y_84"])
                        Dis2=Get_Dis(Data.loc[i,"到达X_84"],Data.loc[i,"到达Y_84"],Data.loc[j,"到达X_84"],Data.loc[j,"到达Y_84"])
                        #从j出发到达i
                        Disji=Get_Dis(Data.loc[j,"出发X_84"],Data.loc[j,"出发Y_84"],Data.loc[i,"到达X_84"],Data.loc[i,"到达Y_84"])
                        #从i出发到达j
                        Disij=Get_Dis(Data.loc[i,"出发X_84"],Data.loc[i,"出发Y_84"],Data.loc[j,"到达X_84"],Data.loc[j,"到达Y_84"])
                        Dis01=Data.loc[i,"行驶距离/m"] #i本身到自己的距离
                        Dis02=Data.loc[j,"行驶距离/m"] #j本身到自己的距离
                        #单独出行速度和合乘速度
                        si=Dis01/ti  #单位是m/minute
                        sj=Dis02/tj
                        snew=(si+sj)/2 #平均速度
                        #计算合乘减少的距离 利用84坐标系
                        #这里进行一个情况的讨论,分别有四种情况
                        delever=False

                        if(O_i<O_j):  #那么肯定先接i
                                start_time = O_i
                                bt1=Dis1/si
                                if(bt1-delta_Begin<time_limit): #说明可以接送
                                    if(Dis02<=Disji): #i->j->j->i
                                        temp=Dis02/snew
                                        if(temp+bt1-time_limit<cal_time(O_i,D_j) and temp+bt1+Dis2/si-time_limit<ti):
                                            Dis_sum=Dis1+Dis02+Dis2
                                            Time_sum=bt1+temp+Dis2/si
                                            dis_hecheng=Dis02
                                            delever=True
                                    if(not delever):  #先接i，先送i ,这时不论距离了，只考虑时间 i->j->i->j
                                        temp=Disji/snew
                                        if(temp+bt1-time_limit<ti and temp+bt1+Dis2/sj-time_limit<cal_time(O_i,D_j)):
                                            Dis_sum=Dis1+Disji+Dis2
                                            Time_sum=bt1+temp+Dis2/sj
                                            dis_hecheng=Disji
                                            delever=True
                                else:
                                    continue  #说明无法合乘
                        else:  #先接j,然后到i
                                start_time = O_j
                                bt1=Dis1/sj
                                if(bt1-delta_Begin<time_limit): #说明可以接送
                                    if(Dis01<=Disij): #先接j，先送i j->i->i->j
                                        temp=Dis01/snew
                                        if(temp+bt1-time_limit<cal_time(O_j,D_i) and temp+bt1+Dis2/sj-time_limit<tj):
                                            Dis_sum=Dis1+Dis01+Dis2
                                            dis_hecheng=Dis01
                                            Time_sum=bt1+temp+Dis2/sj
                                            delever=True
                                    if(not delever):  #j->i->j->i
                                        temp=Disij/snew
                                        if(temp+bt1-time_limit<tj and temp+bt1+Dis2/si-time_limit<cal_time(O_j,D_i)):
                                            Dis_sum=Dis1+Disij+Dis2
                                            dis_hecheng=Disij
                                            Time_sum=bt1+temp+Dis2/si
                                            delever=True
                                else:
                                    continue
                        c1=cost(Dis01,0)
                        c2=cost(Dis02,0)
                        c3=cost(Dis01,1)+cost(Dis02,1)   #1代表合乘
                        cost_jianshao=c1+c2-c3
                        if(delever):  #说明可以进行合乘
                            dis_pingche=(dis_hecheng/Dis_sum)+1
                            dis_jianshao=Dis01+Dis02-Dis_sum #合乘减少的距离
                        #计算合乘减少的时间
                            time_jianshao=ti+tj-Time_sum #合乘减少的时间
                            yanwu1=(Time_sum+temp-ti-tj)/2
                            # 
                            if(leap==0):
                                G.add_edge(i,j,weight=1,another=(dis_jianshao/(Dis01+Dis02),time_jianshao/(ti+tj),cost_jianshao/(c1+c2)),start_time = start_time,time_sum=Time_sum,dis=dis_jianshao,dis1=dis_pingche,yanwu=yanwu1) #以该次合乘的车辆数作为权重
                            elif(leap==1):
                                G.add_edge(i,j,weight=dis_jianshao,another=(dis_jianshao/(Dis01+Dis02),time_jianshao/(ti+tj),cost_jianshao/(c1+c2)),start_time = start_time,time_sum=Time_sum,dis=dis_jianshao,dis1=dis_pingche,yanwu=yanwu1) #以该次合乘的总里程作为权重
                            elif(leap==2):
                                G.add_edge(i,j,weight=time_jianshao,another=(dis_jianshao/(Dis01+Dis02),time_jianshao/(ti+tj),cost_jianshao/(c1+c2)),start_time = start_time,time_sum=Time_sum,dis=dis_jianshao,dis1=dis_pingche,yanwu=yanwu1) #以该次合乘的总时间作为权重
                            elif(leap==3):
                                G.add_edge(i,j,weight=cost_jianshao,another=(dis_jianshao/(Dis01+Dis02),time_jianshao/(ti+tj),cost_jianshao/(c1+c2)),start_time = start_time,time_sum=Time_sum,dis=dis_jianshao,dis1=dis_pingche,yanwu=yanwu1)
    return G

In [20]:
#----------------------------------------相交图转换函数集---------------------------------------#
def ConALL(G1):
    DU_sum=[0]*(G1.number_of_nodes()+1) #G1每个点所连的边在G2的起始位置
    DU_one=[0]*G1.number_of_nodes() #G1每个点所连的边的数量
    G2=nx.Graph() #G2相交图
    G2.add_nodes_from(range(G1.number_of_edges()))
    #建立点
    mark=0 #G2点的索引
    vel=0 #G1点的索引
    for arc in G1.edges():
        G2.nodes[mark]["key"]=arc #记录连接的边作为G2点的属性
        G2.nodes[mark]["weight"]=G1.edges[list((arc))]['weight']#G1边的权重作为G2点的属性
        G2.nodes[mark]["another"]=G1.edges[list((arc))]['another']#G1边的权重作为G2点的属性
        G2.nodes[mark]["start_time"]=G1.edges[list((arc))]['start_time']#G1边的权重作为G2点的属性
        G2.nodes[mark]["time_sum"]=G1.edges[list((arc))]['time_sum']#G1边的权重作为G2点的属性
        G2.nodes[mark]["dis"]=G1.edges[list((arc))]['dis']#G1边的权重作为G2点的属性
        G2.nodes[mark]["dis1"]=G1.edges[list((arc))]['dis1']#G1边的权重作为G2点的属性
        G2.nodes[mark]["yanwu"]=G1.edges[list((arc))]['yanwu']#G1边的权重作为G2点的属性
        while(vel<arc[0]):
            DU_sum[vel+1]=mark #赋值DU_sum
            vel=vel+1
        DU_one[vel]=DU_one[vel]+1 #赋值DU_one
        mark=mark+1
    #建立边
    KONG=[] #度为0的出行
    for node in G1.nodes():
        if G1.degree[node]<=0: #度为0的点加入到KONG
            KONG.append(node)
            continue
        temp=[] #与G1点相连的边在G2的索引
        for i in G1[node]:
            if i<node: #找到小于当前结点的点的具体索引
                ans=0
                for j in G1[i]:
                    if(j<node and j>i):
                        ans=ans+1
                    elif(j==node):
                        break
                bns=ans+DU_sum[i]
                temp.append(bns)
            else:
                break
        for j in range(DU_one[node]): #找到大于当前结点的点的具体索引
            temp.append(DU_sum[node]+j)
        #print(temp)
        for i1 in range(len(temp)):
            for j1 in range(i1+1,len(temp)):
                if(temp[i1]!=temp[j1]):
                    G2.add_edge(temp[i1],temp[j1]) #根据索引进行各点的加边
    return (G2,KONG)

In [21]:
#--------------------------求最小车辆数函数---------------------------#
def Get_MinusCar(G2,D,SE):
    No=[0]*TRIP_NUM     #总体订单序号

    for i in SE:
        u,v=G2.nodes[i]['key']         #筛选可以拼车的订单序号，在总体订单中筛出
        No[u]=1
        No[v]=1
    Order_single_NUM=0
    Order_list=[]
    for i in range(TRIP_NUM):
        if(No[i]==0):
        #用类似时间戳的方式计算时间
            start_time=D.loc[i]["出发时间"].hour*3600+D.loc[i]["出发时间"].minute*60+D.loc[i]["出发时间"].second
            end_time=D.loc[i]["到达时间"].hour*3600+D.loc[i]["到达时间"].minute*60+D.loc[i]["到达时间"].second
            Order_list.append([start_time,end_time,0])
            Order_single_NUM += 1
    #next
    Order_share_NUM = 0
    for i in SE:
        start_time = G2.nodes[i]["start_time"].hour*3600+G2.nodes[i]["start_time"].minute*60+G2.nodes[i]["start_time"].second

        end_time=start_time+G2.nodes[i]["time_sum"]

        Order_list.append([start_time,end_time,0])
        Order_share_NUM+=1
    #find car
    #计算最小车辆数
    car_num=0;
    while(1):
        #找到每一辆车的第一单
        time_min=90000024
        no_min=-1
        for i in range(Order_single_NUM+Order_share_NUM):
            if(Order_list[i][2]==0):
                if(Order_list[i][0]<time_min):
                    no_min=i
                    time_min=Order_list[i][0]

        if(no_min==-1):
            break
        Order_list[no_min][2]=1      #该订单已经标记
        #取出第一单的结束时间
        time_node=Order_list[no_min][1]
        while(1):

            times_node_min=9000909      #距离上一单拼车结束时间节点差值的最小值
            no_node_min = -1            #距离上一单拼车结束时间节点的最小值对应的节点
            for i in range(Order_single_NUM+Order_share_NUM):
                if(Order_list[i][2]==0):
                    if(Order_list[i][0]>time_node):         #新订单的开始时间应在节点之后
                        if(Order_list[i][0]-time_node<times_node_min):          #找最小值
                            times_node_min=Order_list[i][0]-time_node
                            no_node_min=i
                    else:
                        continue
            #找完了收工
            if(no_node_min==-1):

                car_num+=1
                break

            #还没找完接着找
            time_node=Order_list[no_node_min][1]+60*60
            Order_list[no_node_min][2]=1
    return car_num

In [22]:
dis_jianshao=np.zeros(4)
chezailv=np.zeros(4)
pingchelv=np.zeros(4)
raoxingjuli=np.zeros(4)
raoxingbi=np.zeros(4)
yanwu=np.zeros(4)

In [23]:
# #15 15 3下的不同指标的对比
# res=np.zeros([4,4])
# for i in [2]:
#     GD1=Creat_Graph(D,15,3,i)
#     G2,KONG=ConALL(GD1)
#     print("Geneal")
#     SE=ILS_VND(G2,50)
#     d0=0 #绕行比的出行距离 与dis0区分开来
#     dis1=0
#     time1=0
#     cost1=0
#     for j in SE:
#         d0+=D.loc[G2.nodes[j]['key'][0],"行驶距离/m"]
#         d0+=D.loc[G2.nodes[j]['key'][1],"行驶距离/m"]
#         dis1+=G2.nodes[j]['another'][0]
#         time1+=G2.nodes[j]['another'][1]
#         cost1+=G2.nodes[j]['another'][2]
#         dis_jianshao[i]+=G2.nodes[j]['dis']
#         chezailv[i]+=G2.nodes[j]['dis1']
#         yanwu[i]+=G2.nodes[j]['yanwu']
#     car_num=Get_MinusCar(G2,D,SE)
#     res[i][0]=dis1/len(SE)
#     res[i][1]=time1/len(SE)
#     res[i][2]=cost1/len(SE)
#     res[i][3]=car_num/len(D)
#     raoxingjuli[i]=dis_jianshao[i]/len(SE)
#     raoxingbi[i]=dis_jianshao[i]/d0
#     pingchelv[i]+=(len(SE)*2/TRIP_NUM)
#     chezailv[i]=chezailv[i]/len(SE)
#     yanwu[i]=yanwu[i]/len(SE)
#     print(res[i])

In [24]:
# for i in range(len(G2.nodes)):
#         print(G2.nodes[i]['weight'])

In [25]:
#15 15 3下的不同指标的对比
res=np.zeros([4,4])
for i in [0,1,2,3]:
    GD1=Creat_Graph(D,15,3,i)
    G2,KONG=ConALL(GD1)
    SE=ILS_VND(G2,50)
    d0=0 #绕行比的出行距离 与dis0区分开来
    dis1=0
    time1=0
    cost1=0
    for j in SE:
        d0+=D.loc[G2.nodes[j]['key'][0],"行驶距离/m"]
        d0+=D.loc[G2.nodes[j]['key'][1],"行驶距离/m"]
        dis1+=G2.nodes[j]['another'][0]
        time1+=G2.nodes[j]['another'][1]
        cost1+=G2.nodes[j]['another'][2]
        dis_jianshao[i]+=G2.nodes[j]['dis']
        chezailv[i]+=G2.nodes[j]['dis1']
        yanwu[i]+=G2.nodes[j]['yanwu']
    car_num=Get_MinusCar(G2,D,SE)
    res[i][0]=dis1/len(SE)
    res[i][1]=time1/len(SE)
    res[i][2]=cost1/len(SE)
    res[i][3]=car_num/len(D)
    raoxingjuli[i]=dis_jianshao[i]/len(SE)
    raoxingbi[i]=dis_jianshao[i]/d0
    pingchelv[i]+=(len(SE)*2/TRIP_NUM)
    chezailv[i]=chezailv[i]/len(SE)
    yanwu[i]=yanwu[i]/len(SE)
    print(res[i])

[0.2909966  0.3423753  0.21385293 0.18913481]
[0.42779384 0.44628909 0.21471726 0.19114688]
[0.38732355 0.45966982 0.21429573 0.18913481]
[0.3075903  0.3557584  0.21462404 0.18309859]


In [26]:
chezailv

array([1.31639548, 1.42722021, 1.38141168, 1.33919185])

In [27]:
pingchelv

array([0.64386318, 0.58752515, 0.60764588, 0.65191147])

In [28]:
yanwu

array([-4.4216006 , -7.26513391, -8.39511421, -4.54199787])

In [29]:
raoxingjuli

array([ 6696.12323342, 10344.37817365,  8263.61825226,  7238.43916017])

In [30]:
raoxingbi

array([0.33344139, 0.46752455, 0.39366319, 0.34447126])

In [31]:
youhao=(D.loc[:,"行驶距离/m"].sum()-dis_jianshao)*8*7.8/1000/100
youhao

array([2587.80280403, 2313.92951852, 2477.71258145, 2524.62440983])

In [32]:
# import matplotlib.pyplot as plt
# plt.rcParams["font.sans-serif"]=["SimHei"]
# plt.rcParams["axes.unicode_minus"]=False
# my_x_ticks=["Vel_Num","Dis","Time","Cost"]
# N=["Reduced vel_num",'Reduced distance', 'Reduced time',"Reduced Cost"]

In [33]:
# import plotly.graph_objects as go
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=my_x_ticks,
#         y=res[0],
#         mode="markers+lines",
#         name=N[0],   
# ))
# fig.add_trace(
#     go.Scatter(
#         x=my_x_ticks,
#         y=res[1],
#         mode="markers+lines",
#         name=N[1],
# ))
# fig.add_trace(
#     go.Scatter(
#         x=my_x_ticks,
#         y=res[2],
#         mode="markers+lines",
#         name=N[2], 
# ))
# fig.add_trace(
#     go.Scatter(
#         x=my_x_ticks,
#         y=res[3],
#         mode="markers+lines",
#         name=N[3], 
# ))
# fig.update_layout(
#     title="Comparison chart under different indicators",
#     xaxis=dict(title="Indicators", nticks=13),
#     yaxis=dict(title="Percent", nticks=11, rangemode="tozero", range=(0,0.4)),
#     width=1000,
#     height=500,
# #     template='plotly_dark'
# )
# fig.show()


In [ ]:
#以time_limit为变量进行灵敏度分析
td=range(0,16)
res1=np.zeros([4,len(td)])
for i in range(1,16):
    GD1=Creat_Graph(D,i,3,3) #100数据的路网 以路程为权重
    G2,KONG=ConALL(GD1)
    SE=ILS_VND(G2,50)
    time1=0
    dis1=0
    cost1=0
    for j in SE:
        time1+=G2.nodes[j]['another'][0]
        dis1+=G2.nodes[j]['another'][1]
        cost1+=G2.nodes[j]['another'][2]
    car_num=Get_MinusCar(G2,D,SE)
    res1[0][i]=dis1/len(SE)
    res1[1][i]=time1/len(SE)
    res1[2][i]=cost1/len(SE)
    res1[3][i]=car_num/len(D)
    print(res1[0][i],res1[1][i],res1[2][i],res1[3][i])

In [ ]:
# import plotly.graph_objects as go
# X=np.arange(1,16)
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=X,
#         y=res1[0],
#         mode="markers+lines+text",
#         name=N[0],   
#         line_shape='spline'
# ))
# fig.add_trace(
#     go.Scatter(
#         x=X,
#         y=res1[1],
#         mode="markers+lines",
#         name=N[1],
#         line_shape='spline',
#         marker={"symbol":"star","size":10}
# ))
# fig.add_trace(
#     go.Scatter(
#         x=X,
#         y=res1[2],
#         mode="markers+lines",
#         name=N[2], 
#         text=res1[2],
#         marker={"symbol":"x","size":10}
# ))
# fig.update_layout(
#     title="Multiplication weighted by TimeLimit
#     xaxis=dict(title="Time limit", nticks=16),
#     yaxis=dict(title="Percent", nticks=11, rangemode="tozero", range=(0,0.5)),
#     width=1000,
#     height=500,
#     template='simple white'
# )
# # 通过位置参数来改变
# fig.update_layout(legend=dict(
#     yanchor="top",  # y轴顶部
#     y=0.99,
#     xanchor="left",  # x轴靠左
#     x=0.01
# ))
# fig.show()
